In [1]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, mean_squared_error, roc_curve
# sklearn.metrics -  scikit-learn 패키지 중 모델 평가에 사용되는 모듈
## lassification_report - sklearn.metrics 패키지의 모듈 중 주요 분류 측정 항목을 보여주는 보고서 모듈
## confusion_matrix - 분류의 정확성을 평가하기 위한 오차행렬 계산 모듈

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# sklearn.model_selection - scikit-learn 패키지 중 클래스를 나눌 때, 함수를 통해 train/test셋을 나눌 때, 모델 검증에 쓰는 서브 패키지
## train_test_split - sklearn.model_selection 패키지의 모듈중 배열 또는 행렬을 임의의 훈련 및 테스트 셋으로 분할하는 모듈

from sklearn.tree import DecisionTreeClassifier
# sklearn.tree - scikit-learn 패키지 중 분류 및 회귀를 위한 의사결정 트리 기반 모델이있는 서브 패키지
## DecisionTreeClassifier - sklearn.tree 패키지의 모듈 중 의사결정 트리 분류 모듈

from IPython.display import Image
# Iphython.display - Ipython 내에 정보를 보여주는 도구용도의 공용 API
## Image - IPython.display API 모듈 중 원시(raw)데이터가 있는 png나 jpeg등의 이미지 객체를 만드는 모듈

from sklearn.preprocessing import StandardScaler, OneHotEncoder
# StandardScaler - 평균을 제거하고 데이터를 단위 분산으로 조정하나 이상치가 있다면 데이터의 확산이 매우 달라짐

import pandas as pd
# pandas - 데이터를 구조화된 형식으로 가공 및 분석할 수 있도록 자료구조를 제공하는 패키지

import numpy as np
# numpy -  Numerical Python으로 고성능 계산이나 데이터 분석에 유용한 패키지


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import seaborn as sns
from patsy import dmatrices
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import datasets
from sklearn.ensemble import AdaBoostRegressor

In [2]:
data_A = pd.read_csv("train.csv")
data_A.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [3]:
# 범주형 변수 중 str형식으로 되어있는 변수 0,1,2로 변경
# 범주형 변수: Gender, Driving_License, Region_Code, Previously_Insured, Vehicle_Age, Vehicle_Damage, Response, Policy_Sales_Channel
# Gender: Male-1, Female-0
data_A.loc[data_A["Gender"] == "Female","Gender"] = 0
data_A.loc[data_A["Gender"] == "Male","Gender"] = 1
# Vehicle_Age: <1-0, 1-2:1, >2:2
data_A.loc[data_A["Vehicle_Age"] == "< 1 Year","Vehicle_Age"] = 0
data_A.loc[data_A["Vehicle_Age"] == "1-2 Year","Vehicle_Age"] = 1
data_A.loc[data_A["Vehicle_Age"] == "> 2 Years","Vehicle_Age"] = 2
# Vehicle_Damage: Yes-1, No-1
data_A.loc[data_A["Vehicle_Damage"] == "No","Vehicle_Damage"] = 0
data_A.loc[data_A["Vehicle_Damage"] == "Yes","Vehicle_Damage"] = 1

In [4]:
data_A.info()
data_A.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,1,44,1,28.0,0,2,1,40454.0,26.0,217,1
1,2,1,76,1,3.0,0,1,0,33536.0,26.0,183,0
2,3,1,47,1,28.0,0,2,1,38294.0,26.0,27,1
3,4,1,21,1,11.0,1,0,0,28619.0,152.0,203,0
4,5,0,29,1,41.0,1,0,0,27496.0,152.0,39,0


In [5]:
# 데이터 타입 변경
data_A['Gender'] = data_A['Gender'].astype('category')
data_A['Age'] = data_A['Age'].astype('float')
data_A['Driving_License'] = data_A['Driving_License'].astype('category')
data_A['Region_Code'] = data_A['Region_Code'].astype('category')
data_A['Previously_Insured'] = data_A['Previously_Insured'].astype('category')
data_A['Vehicle_Age'] = data_A['Vehicle_Age'].astype('category')
data_A['Vehicle_Damage'] = data_A['Vehicle_Damage'].astype('category')
data_A['Annual_Premium'] = data_A['Annual_Premium'].astype('float')
data_A['Policy_Sales_Channel'] = data_A['Policy_Sales_Channel'].astype('category')
data_A['Vintage'] = data_A['Vintage'].astype('float')
data_A['Response'] = data_A['Response'].astype('category')

In [6]:
# 변경된 데이터 타입 확인
data_A.dtypes

id                         int64
Gender                  category
Age                      float64
Driving_License         category
Region_Code             category
Previously_Insured      category
Vehicle_Age             category
Vehicle_Damage          category
Annual_Premium           float64
Policy_Sales_Channel    category
Vintage                  float64
Response                category
dtype: object

In [7]:
#음수값 없게  minmaxscaler 사용 (0~1)
from sklearn.preprocessing import MinMaxScaler
data_B = data_A
data_B['Age'] = (data_B['Age']-data_B['Age'].min())/(data_B['Age'].max()-data_B['Age'].min())
data_B['Annual_Premium'] = (data_B['Annual_Premium']-data_B['Annual_Premium'].min())/(data_B['Annual_Premium'].max()-data_B['Annual_Premium'].min())
data_B['Vintage'] = (data_B['Vintage']-data_B['Vintage'].min())/(data_B['Vintage'].max()-data_B['Vintage'].min())

In [8]:
res = data_B['Response']
res

0         1
1         0
2         1
3         0
4         0
         ..
381104    0
381105    0
381106    0
381107    0
381108    0
Name: Response, Length: 381109, dtype: category
Categories (2, int64): [0, 1]

In [9]:
# 정규화된 데이터 확인 및 column drop
data_B = data_B.drop(['id'], axis = 1)
data_B = data_B.drop(['Response'], axis = 1)
data_B

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,1,0.369231,1,28.0,0,2,1,0.070366,26.0,0.716263
1,1,0.861538,1,3.0,0,1,0,0.057496,26.0,0.598616
2,1,0.415385,1,28.0,0,2,1,0.066347,26.0,0.058824
3,1,0.015385,1,11.0,1,0,0,0.048348,152.0,0.667820
4,0,0.138462,1,41.0,1,0,0,0.046259,152.0,0.100346
...,...,...,...,...,...,...,...,...,...,...
381104,1,0.830769,1,26.0,1,1,0,0.051234,26.0,0.269896
381105,1,0.153846,1,37.0,1,0,0,0.069551,152.0,0.418685
381106,1,0.015385,1,30.0,1,0,0,0.060439,160.0,0.522491
381107,0,0.738462,1,14.0,0,2,1,0.078110,124.0,0.221453


In [10]:
# train, validation, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(data_B, res, test_size=0.3, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(data_B, res, test_size=0.2/0.7, random_state=0)

In [11]:
# Smote 샘플링
smote_nc = SMOTENC([0, 2, 3, 4, 5, 6, 8], random_state=0)
X_resampled, y_resampled = smote_nc.fit_resample(X_train,list(y_train))

# Bagging Ensemble 

In [12]:
# 배깅(페이스팅) 모델링(bootstrap: 배깅True(보통 선호), 페이스팅False)
BG = BaggingClassifier(DecisionTreeClassifier(), n_estimators=100, max_samples=100, bootstrap=True, n_jobs=-1, oob_score=True,random_state=0)
BG.fit(X_resampled, y_resampled)
y_pred_BG = BG.predict(X_val)

In [13]:
# 혼동행렬
print("Train data\n",confusion_matrix(y_train, BG.predict(X_train)))
print("validation data\n",confusion_matrix(y_val,y_pred_BG))

Train data
 [[161942  76944]
 [  3216  30118]]
validation data
 [[64522 30991]
 [ 1259 12117]]


In [14]:
# 정확도, 재현율, f1 점수
print(classification_report(y_val, y_pred_BG))

              precision    recall  f1-score   support

           0       0.98      0.68      0.80     95513
           1       0.28      0.91      0.43     13376

    accuracy                           0.70    108889
   macro avg       0.63      0.79      0.61    108889
weighted avg       0.89      0.70      0.75    108889



# Adaboost Ensemble

In [ ]:
# 아다부스트 모델링
AD = AdaBoostClassifier(DecisionTreeClassifier(max_depth=50), n_estimators=200, learning_rate=1)
AD.fit(X_resampled, y_resampled)
y_pred_AD = AD.predict(X_val)

In [ ]:
# 혼동행렬
print("Train data\n",confusion_matrix(y_resampled, AD.predict(X_resampled)))
print("Validation data\n",confusion_matrix(y_val, y_pred_AD))

In [ ]:
# 정확도, 재현율, f1 점수
print(classification_report(y_val, y_pred_AD))

# Random Forest

In [ ]:
# 랜덤포레스트 모델링(criterion:분류기준,n_estimators:나무 생성 수,min_samples_split:,max_features:최대 고려 특성 수,oob_score:정확도)
RF = RandomForestClassifier(criterion = 'entropy',n_estimators=20,min_samples_split=20,max_features=4,max_depth=12,random_state=0)
RF.fit(X_resampled, y_resampled)
y_pred_RF = RF.predict(X_val)


In [ ]:
# 혼동행렬
print("Train data\n",confusion_matrix(y_resampled, RF.predict(X_resampled)))
print("Validation data\n",confusion_matrix(y_val, y_pred_RF))

In [ ]:
# 정확도, 재현율, f1 점수
print(classification_report(y_val, y_pred_RF))

# Logistic Regression

In [ ]:
# 로지스틱 리그레션 모델링
LR = LogisticRegression(C=0.001,penalty="l2")
LR.fit(X_resampled,y_resampled)
y_pred_LR = LR.predict(X_val)

In [13]:
# 혼동행렬
print("Train data","\n",confusion_matrix(y_resampled, LR.predict(X_resampled)))
print("Validation data","\n",confusion_matrix(y_val, y_pred_LR))

Train data 
 [[141255  97631]
 [  3257 235629]]
Validation data 
 [[56307 39206]
 [  296 13080]]


In [14]:
# 정확도, 재현율, f1 점수
print(classification_report(y_val, y_pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.59      0.74     95513
           1       0.25      0.98      0.40     13376

    accuracy                           0.64    108889
   macro avg       0.62      0.78      0.57    108889
weighted avg       0.90      0.64      0.70    108889



# Lightgbm

In [ ]:
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(n_estimators=100, subsample=0.8, n_jobs=-1, boost_from_average=False) 
lgbm_clf.fit(X_resampled, y_resampled)
lgbm_pred = lgbm_clf.predict(X_val)

In [ ]:
print("Train data","\n",confusion_matrix(y_resampled, lgbm_clf.predict(X_resampled)))
print("Validation data","\n",confusion_matrix(y_val, lgbm_pred))

In [ ]:
print(classification_report(y_val, lgbm_pred))

# KNN

In [ ]:
# KNN 모델링(n_neighbors: 이웃 수, p: 차원, metric: 거리 측정방법)
KN = KNeighborsClassifier(n_neighbors=4, p=3, metric='minkowski')
KN.fit(X_resampled, y_resampled)
y_pred_KN = KN.predict(X_val)

In [ ]:
# 혼동행렬
print("Train data","\n",confusion_matrix(y_resampled, KN.predict(X_resampled)))
print("Validation data","\n",confusion_matrix(y_val, y_pred_KN))

In [ ]:
# 정확도, 재현율, f1 점수
print(classification_report(y_val, y_pred_KN))

# Bayes'

In [25]:
#naive bayes' 모델링
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB().fit(X_resampled,y_resampled)
y_pred_nb = model.predict(X_val)

In [26]:
#혼돈행렬
print("Train data","\n",confusion_matrix(y_resampled, model.predict(X_resampled)))
print("Validation data","\n",confusion_matrix(y_val, y_pred_nb))

Train data 
 [[173477  65409]
 [ 90343 148543]]
Validation data 
 [[69293 26220]
 [ 5147  8229]]


In [27]:
# 정확도, 재현율, f1 점수
print(classification_report(y_val, y_pred_nb))

              precision    recall  f1-score   support

           0       0.93      0.73      0.82     95513
           1       0.24      0.62      0.34     13376

    accuracy                           0.71    108889
   macro avg       0.58      0.67      0.58    108889
weighted avg       0.85      0.71      0.76    108889



# 선정된 모델 최적 parameter 확인

In [12]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [10, 100],
           'max_depth' : [30, 60, 80],
           'min_samples_leaf' : [6, 8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
rf_clf = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_resampled, y_resampled)

print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 80, 'min_samples_leaf': 6, 'min_samples_split': 8, 'n_estimators': 100}
최고 예측 정확도: 0.8614


# 선정된 모델로 test data 예측 수행
### model: random forest

In [13]:
y_pred_RF = grid_cv.predict(X_test)

In [14]:
# 혼동행렬
print("Test data\n",confusion_matrix(y_test, y_pred_RF))

Test data
 [[79965 20276]
 [ 4197  9895]]


In [15]:
# 정확도, 재현율, f1 점수
print(classification_report(y_test, y_pred_RF))

              precision    recall  f1-score   support

           0       0.95      0.80      0.87    100241
           1       0.33      0.70      0.45     14092

    accuracy                           0.79    114333
   macro avg       0.64      0.75      0.66    114333
weighted avg       0.87      0.79      0.82    114333

